### Import libraries and dependencies

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

pd.options.display.float_format = '{:,.0f}'.format

In [2]:
def excel_parser(file_path):
    '''
    This function takes the file path and read multiple sheets in it while parsing them through the necessary 
    removal from the excel cells. Then it writes each sheet as a dataframe and collects it in dictionary.
    '''
    xl = pd.ExcelFile(file_path)
    xl_sheets = xl.sheet_names[1:-2]

    all_sheets_df = []
    for sheet in xl_sheets:
        sheets_dict = {}
        df_name = 'df_'+sheet.replace(" ","")
        
        df = xl.parse(sheet, skiprows=14, convert_float=False, header=None)
        
        for i in df.iloc[:1].columns:
            if type(df.iloc[:1][i][0])==str:
                
                # replace "International migrant stock at mid-year by age (both sexes)" by 'both_sexes' in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('International migrant stock at mid-year by age (both sexes)','both_sexes')
                
                # replace "International migrant stock at mid-year by age " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('International migrant stock at mid-year by age ','')
                
                # replace "Total population of both sexes at mid-year (thousands)g" by "both_sexes" in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Total population of both sexes at mid-year (thousands)g','both_sexes')
                
                # replace "Total female population at mid-year (thousands)" by "(female)" in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Total female population at mid-year (thousands)','female')
                
                # replace "Total male population at mid-year (thousands)" by "(male)" in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Total male population at mid-year (thousands)','male')
                
                # replace "International migrant stock as percentage of the total population " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('International migrant stock as percentage of the total population ','')
                
                # replace "Percentage distribution of the international migrant stock " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Percentage distribution of the international migrant stock ','')
                
                # replace "Female migrants as a percentage of the international migrant stock" by "FemaleMigr_%_IMS" in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Female migrants as a percentage of the international migrant stock','FemaleMigr_%_IMS')
                
                # replace "International migrant stock at mid-year " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('International migrant stock at mid-year ','')

                # replace "Annual rate of change of the migrant stock " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Annual rate of change of the migrant stock ','')

                # replace "Estimated refugee stock (including asylum seekers) at mid-year (both sexes)" by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Estimated refugee stock (including asylum seekers) at mid-year (both sexes)','both_sexes')

                # replace "Refugees and asylum seekers as a percentage of the international migrant stock" by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Refugees and asylum seekers as a percentage of the international migrant stock','RefAsylum_%_IMS')

                # replace "Annual rate of change of the refugee stock (including asylum seekers)" by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Annual rate of change of the refugee stock (including asylum seekers)','ARchange_ofRefAsylum')

                # remove "(", ")" from the row that is going to be header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('(','').replace(')','')

        #Then first forward fill the NaNs in the first two rows (thus propagating 'Auto loan', for example).
        df.iloc[0:2] = df.iloc[0:2].fillna(method='ffill', axis=1)
        #Next fill in the remaining NaNs with empty strings:
        df.iloc[0:2] = df.iloc[0:2].fillna('')

        #Now join the two rows together with '_' and assign that as the column level values:
        df.columns = df.iloc[0:2].apply(lambda x: '_'.join([str(y) for y in x if y]), axis=0)

        #And finally, remove the first two rows:
        df = df.iloc[2:]
        
        #Remove the rows woth all NaN values
        df = df.dropna(how='all')
        
        #Convert year from float to integer
        if df.columns.str.contains("Year").any()==True:
            df['Year'] = df['Year'].astype(int)
        
        sheets_dict = {df_name: df}
        all_sheets_df.append(sheets_dict)
    return(all_sheets_df)

## I. Read Migrant Stock Datasets

### i. Reading dataset of _UN Migrant Stock By Age And Sex 2019_

In [3]:
#Read the file
file_path_AgeSex = "./data/UN_data/UN_MigrantStockByAgeAndSex_2019.xlsx"
MigrStock_ageSex = excel_parser(file_path_AgeSex)

df_Table1_AgeSex = MigrStock_ageSex[0]['df_Table1'].reset_index(drop=True)
df_Table2_AgeSex = MigrStock_ageSex[1]['df_Table2'].reset_index(drop=True)
df_Table3_AgeSex = MigrStock_ageSex[2]['df_Table3'].reset_index(drop=True)
df_Table4_AgeSex = MigrStock_ageSex[3]['df_Table4'].reset_index(drop=True)
df_Table5_AgeSex = MigrStock_ageSex[4]['df_Table5'].reset_index(drop=True)

df_ANNEX_AgeSex = pd.ExcelFile(file_path_AgeSex).parse('ANNEX', skiprows=15, convert_float=False).drop(['Index'], axis=1)


In [4]:
df_Table1_AgeSex.head()

,Year,Sort\norder,"Major area, region, country or area of destination",Notes,Code,Type of data a,both_sexes_0-4,both_sexes_5-9,both_sexes_10-14,both_sexes_ 15-19,...,female_35-39,female_40-44,female_45-49,female_50-54,female_55-59,female_60-64,female_65-69,female_70-74,female_75+,female_Total
0,1990,1990001,WORLD,NaN,900,NaN,"4,767,961","6,176,471","7,729,769","9,753,361",...,"6,863,925","6,193,801","5,202,241","4,431,846","3,993,850","3,702,649","2,977,746","2,543,862","4,929,603","75,349,784"
1,1990,1990002,UN development groups,NaN,NaN,NaN,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,1990,1990003,More developed regions,b,901,NaN,"1,218,097","2,336,625","3,515,354","4,778,797",...,"4,114,643","3,845,971","3,278,864","2,791,841","2,524,115","2,334,839","1,825,701","1,567,961","3,036,174","42,340,418"
3,1990,1990004,Less developed regions,c,902,NaN,"3,549,864","3,839,846","4,214,415","4,974,564",...,"2,749,282","2,347,830","1,923,377","1,640,005","1,469,735","1,367,810","1,152,045","975,901","1,893,429","33,009,366"
4,1990,1990005,Least developed countries,d,941,NaN,"1,016,860","1,130,722","1,102,576","1,206,354",...,"370,929","289,525","221,254","190,213","156,055","141,991","78,103","51,194","70,448","5,509,988"


### ii. Reading dataset of _UN Migrant Stock By Origin And Destination 2019_

In [5]:
#Read the file
file_path_OriginDestination = "./data/UN_data/UN_MigrantStockByOriginAndDestination_2019.xlsx"
MigrStock_originDestination = excel_parser(file_path_OriginDestination)

df_Table1_OriginDest = MigrStock_originDestination[0]['df_Table1'].reset_index(drop=True)
df_Table2_OriginDest = MigrStock_originDestination[1]['df_Table2'].reset_index(drop=True)
df_Table3_OriginDest = MigrStock_originDestination[2]['df_Table3'].reset_index(drop=True)

df_ANNEX_OriginDest = pd.ExcelFile(file_path_OriginDestination).parse('ANNEX', skiprows=15, convert_float=False).drop(['Index'], axis=1)


In [6]:
df_Table1_OriginDest.head()

,Year,Sort\norder,"Major area, region, country or area of destination",Notes,Code,Type of data a,Type of data a_Total,Type of data a_Other South,Type of data a_Other North,Country or area of origin_Afghanistan,...,Country or area of origin_Uruguay,Country or area of origin_Uzbekistan,Country or area of origin_Vanuatu,Country or area of origin_Venezuela (Bolivarian Republic of),Country or area of origin_Viet Nam,Country or area of origin_Wallis and Futuna Islands,Country or area of origin_Western Sahara,Country or area of origin_Yemen,Country or area of origin_Zambia,Country or area of origin_Zimbabwe
0,1990,1990001,WORLD,NaN,900,NaN,"153,011,473","6,548,526","2,366,800","6,823,350",...,"237,486","1,428,020","5,060","185,946","1,237,873","6,484","168,239","455,492","85,203","204,365"
1,1990,1990002,UN development groups,NaN,NaN,NaN,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,1990,1990003,More developed regions,b,901,NaN,"82,767,216","3,385,103","1,077,179","119,386",...,"56,838","1,078,563","1,017","114,991","1,085,310",884,333,"11,457","26,062","40,957"
3,1990,1990004,Less developed regions,c,902,NaN,"70,244,257","3,163,423","1,289,621","6,703,964",...,"180,648","349,457","4,043","70,955","152,563","5,600","167,906","444,035","59,141","163,408"
4,1990,1990005,Least developed countries,d,941,NaN,"11,060,221","482,753","239,756",0,...,286,"2,027",9,"2,510","71,579",0,0,357,"26,254","75,122"


### iii. Reading dataset of _UN Migrant Stock Total 2019_

In [7]:
#Read the file
file_path_StockTotal = "./data/UN_data/UN_MigrantStockTotal_2019.xlsx"
MigrStock_StockTotal = excel_parser(file_path_StockTotal)

df_Table1_StockTotal = MigrStock_StockTotal[0]['df_Table1'].reset_index(drop=True)
df_Table2_StockTotal = MigrStock_StockTotal[1]['df_Table2'].reset_index(drop=True)
df_Table3_StockTotal = MigrStock_StockTotal[2]['df_Table3'].reset_index(drop=True)
df_Table4_StockTotal = MigrStock_StockTotal[3]['df_Table4'].reset_index(drop=True)
df_Table5_StockTotal = MigrStock_StockTotal[4]['df_Table5'].reset_index(drop=True)
df_Table6_StockTotal = MigrStock_StockTotal[5]['df_Table6'].reset_index(drop=True)

df_ANNEX_StockTotal = pd.ExcelFile(file_path_StockTotal).parse('ANNEX', skiprows=15, convert_float=False).drop(['Index'], axis=1)


In [8]:
df_Table1_StockTotal.head()

,Sort\norder,"Major area, region, country or area of destination",Notes,Code,Type of data a,both sexes_1990.0,both sexes_1995.0,both sexes_2000.0,both sexes_2005.0,both sexes_2010.0,...,male_2010.0,male_2015.0,male_2019.0,female_1990.0,female_1995.0,female_2000.0,female_2005.0,female_2010.0,female_2015.0,female_2019.0
0,1,WORLD,NaN,900,NaN,"153,011,473","161,316,895","173,588,441","191,615,574","220,781,909",...,"114,061,680","128,863,389","141,488,004","75,349,784","79,630,779","85,559,220","93,754,736","106,720,229","119,997,907","130,154,101"
1,2,UN development groups,NaN,NaN,NaN,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,3,More developed regions,b,901,NaN,"82,767,216","92,935,095","103,961,989","116,687,616","130,613,460",...,"63,408,858","67,824,389","73,765,353","42,340,418","47,557,507","53,160,091","59,609,215","67,204,602","72,818,928","78,303,908"
3,4,Less developed regions,c,902,NaN,"70,244,257","68,381,800","69,626,452","74,927,958","90,168,449",...,"50,652,822","61,039,000","67,722,651","33,009,366","32,073,272","32,399,129","34,145,521","39,515,627","47,178,979","51,850,193"
4,5,Least developed countries,d,941,NaN,"11,060,221","11,681,777","10,063,948","9,833,150","10,432,671",...,"5,185,496","6,784,461","8,086,158","5,509,988","5,857,700","5,030,016","4,845,613","5,247,175","6,846,888","8,202,865"


## II. Read Migrant Flow Datasets

### i. migflows_allcountries_1990_2015.csv Data Details:

**Row for each migration corridor - period combination (200 origins x 200 destinations x 5 periods = 200,000).**

- year0 - first year of five year period
- orig - origin ISO three letter country code
- dest - destination ISO three letter country

**Columns for estimates based on the following migration flow estimation methods:**

Stock Differencing Approaches: 
- sd_drop_neg - https://doi.org/10.1016/j.jdeveco.2009.11.004
- sd_rev_neg - https://doi.org/10.1111/sjoe.12098

Migration Rate Approach:
- mig_rate - https://doi.org/10.1002/9781118937464.ch7]

Demographic Accounting Approaches:
- da_min_open - https://doi.org/10.4054/DemRes.2013.28.18
- da_min_closed - https://doi.org/10.1111/imre.12327
- da_pb_closed - https://doi.org/10.1073/PNAS.1722334116

In [9]:
#Read the file
df_MigrFlows = pd.read_csv("./data/Abel/migflows_allcountries_1990_2015.csv")
df_MigrFlows.head()

,year0,orig,dest,sd_drop_neg,sd_rev_neg,mig_rate,da_min_open,da_min_closed,da_pb_closed
0,1990,BDI,BDI,0,0,0,0,0,0
1,1990,COM,BDI,0,0,0,0,0,0
2,1990,DJI,BDI,0,0,0,0,0,0
3,1990,ERI,BDI,0,0,0,0,0,83
4,1990,ETH,BDI,0,0,0,0,0,2


### ii. migflows_allcountries_gender_separated_1990_2015.csv Data Details

**Column details:**
- stock: stock data source used for the estimated flow (un12, un13, un15 or wb11)
- demo: demographic data source used for the estimated flow (wpp2010, wpp2012, wpp2015)
- sex: gender of the estimated flow
- year0: first year of the period of the estimated flow (ranging from 1960 to 2010, and varying in lengths depending on the stock and demo data, as described in the paper)
- interval: the length of the period of the estimated flow 
- orig: ISO 3166-1 alpha-3 letter code for the origin country of the estimated flow
- dest: ISO 3166-1 alpha-3 letter code for the destination country of the estimated flow
- orig_code: ISO 3166-1 numeric code for the origin country of the estimated flow
- dest_code: ISO 3166-1 numeric code for the destination country of the estimated flow
- flow: estimated flow

More details on the country codes:
- https://www.iso.org/iso-3166-country-codes.html
- https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3
- https://en.wikipedia.org/wiki/ISO_3166-1_numeric

More details on tidy data:
- http://vita.had.co.nz/papers/tidy-data.pdf

In [10]:
#Read the file
df_MigrFlowSex = pd.read_csv("./data/Abel/migflows_allcountries_gender_separated_1990_2015.csv")
df_MigrFlowSex.head()

,stock,demo,sex,year0,interval,orig,dest,orig_code,dest_code,flow
0,un12,wpp2010,b,1990,10,ABW,ABW,533,533,0
1,un12,wpp2010,b,1990,10,ABW,AFG,533,4,0
2,un12,wpp2010,b,1990,10,ABW,AGO,533,24,0
3,un12,wpp2010,b,1990,10,ABW,ALB,533,8,0
4,un12,wpp2010,b,1990,10,ABW,ANT,533,530,0


## III. Read Refugee population by country or territory of asylum Datasets

Useful information about some terms in the Refugee-Asylum Dataset:

- Refugees are people who are recognized as refugees under the 1951 Convention Relating to the Status of Refugees or its 1967 Protocol, the 1969 Organization of African Unity Convention Governing the Specific Aspects of Refugee Problems in Africa, people recognized as refugees in accordance with the UNHCR statute, people granted refugee-like humanitarian status, and people provided temporary protection. 

- Asylum seekers--people who have applied for asylum or refugee status and who have not yet received a decision or who are registered as asylum seekers--are excluded. 

- Palestinian refugees are people (and their descendants) whose residence was Palestine between June 1946 and May 1948 and who lost their homes and means of livelihood as a result of the 1948 Arab-Israeli conflict. Country of asylum is the country where an asylum claim was filed and granted.


__Source:__ Data before 2018 are from United Nations High Commissioner for Refugees ( UNHCR ), Statistics Database, Statistical Yearbook and data files, complemented by statistics on Palestinian refugees under the mandate of the UNRWA as published on its website. Data from UNHCR are available online at: unhcr.org/en-us/figures-at-a-glance.html. The data for 2018 are from UNHCR Global Trends report 2018 and World Bank's estimates ( for Palestinian refugees in the UNRWA areas of operation ).


In [11]:
#Read the file
file_path_Refugee = "./data/TheWorldBankRefugee/API_SM.POP.REFG_DS2_en_excel_v2_890428.xls"
df_Refugee = pd.ExcelFile(file_path_Refugee).parse('Data', skiprows=3)

In [12]:
#Remove all the NaN columns that have no value for the further analysis from the data 
# find all the columns which have all NaN values in it
nan_columns = df_Refugee.columns[df_Refugee.isnull().all()]#.tolist()
df_Refugee = df_Refugee.drop(nan_columns, axis=1)

df_Refugee.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1990,1991,1992,1993,1994,1995,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,Refugee population by country or territory of ...,SM.POP.REFG,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,1,nan,2,1,nan,nan
1,Afghanistan,AFG,Refugee population by country or territory of ...,SM.POP.REFG,50,38,"60,025","32,132","19,131","19,605",...,37,"6,434","3,009","16,187","16,863","300,423","257,554","59,771","75,927","72,231"
2,Angola,AGO,Refugee population by country or territory of ...,SM.POP.REFG,"11,557","11,022","11,002","10,878","10,686","10,884",...,"14,734","15,155","16,223","23,413","23,783","15,474","15,555","15,555","41,109","39,865"
3,Albania,ALB,Refugee population by country or territory of ...,SM.POP.REFG,nan,nan,"3,000","3,000","3,000","4,720",...,70,76,82,86,93,104,104,138,89,131
4,Andorra,AND,Refugee population by country or territory of ...,SM.POP.REFG,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
